In [1390]:
# For simplifying expressions
from sympy import symbols, Eq, simplify, And, Or, Not
from sympy.logic import simplify_logic
# from sympy.logic.boolalg import to_dnf

# import sympy.logic.boolalg as b
# print(dir(b))

import time


In [1391]:
# Some Definitions of Symbols (0,1,AND,OR)
_false = '0'
_true = '1'
_and = "AND"
_or = "OR"


In [1392]:
# sympy example use part 0

X = [symbols("X[0]"),symbols("X[1]"),symbols("X[2]")]

A,B,C,D,E,F,G = symbols("A B C D E F G")

expr = E | (A & E) | (B & E) | (C & True) | (D & (F & G))

# expr = expr.subs({A : 3})

print(simplify(expr))  # still returns (A & B) | (A & ~B)

C | E | (D & F & G)


In [1393]:
# sympy example use part 1

# v = ['(', '(', 'x[0]', '==', 1, ')', 'AND', '(', '(', '(', 'x[1]', '==', 0, ')', 'AND', '(', 'l[1][0]', ')', ')', 'OR', '(', '(', 'x[2]', '==', 0, ')', 'AND', '(', 'l[1][0]', ')', ')', 'OR', '(', '(', 'x[3]', '==', 0, ')', 'AND', '(', 'l[1][0]', ')', ')', ')', ')']

In [1394]:
# sympy example use part 2

# l = [[[] for t in range(10)] for w in range(10)]

# for t in range(10):
#   for w in range(10):
#     l[w][t] = symbols(f"l[{w}][{t}]")

# x = [[] for i in range(4)]

# for i in range(4):
#   x[i] = symbols(f"x[{i}]")

# v_prime = []

# i = 0
# while i < len(v):
#   if v[i] == "AND": v_prime.append("&"); i+=1 #; print(v_prime); print(1)
#   elif v[i] == "OR": v_prime.append("|"); i+=1 #; print(v_prime); print(2)
#   elif v[i][0] == "x":
#     v_prime.append(f"Eq({v[i]},{v[i+2]})")
#     i+=3 #; print(v_prime); print(3)
#   else:
#     v_prime.append(v[i]); i+=1 #; print(v_prime); print(4)

# # print(v_prime)
# print(" ".join(v_prime))
# print()

# expr = eval(" ".join(v_prime))

# expr2 = expr.subs({l[1][0] : Eq(x[0],0)})

# simp = simplify(simplify_logic(expr2))

# print(expr)
# print()
# print(expr2)
# print()
# print(simp)
# print()
# print(str(expr))

In [1395]:
def convert_sympy_str_to_vector(s):
    
    # its for the cases of "Eq(X[w], 0)" to turn into "Eq(X[w],0)"
    s2 = s.replace(", ", ",")
    # takes everything separated by a " " apart and puts into v
    v = s2.split(" ")

    # Vector for the translation of v from the sympy symbols
    v_from_sympy = []

    # Translating from SymPys version to ours
    i = 0
    while i < len(v):
        if v[i] == 'False': v_from_sympy.append(_false); i+=1
        elif v[i] == 'True': v_from_sympy.append(_true); i+=1
        elif v[i] == '&': v_from_sympy.append(_and); i+=1
        elif v[i] == '|': v_from_sympy.append(_or); i+=1
        else: 
            # turns all X to x and L to l
            v_i_translation = v[i].translate(str.maketrans({"X": "x", "L": "l"}))
            v_from_sympy.append(v_i_translation); i+=1

    return v_from_sympy

In [1396]:
def clean_vector_sympy(v, t_max, w_max, b_max):
    # ---------------- Maybe put all this OUTSIDE somewhere ----------------
    L = [[[] for t in range(t_max+1)] for w in range(w_max)]
    X = [[] for b in range(b_max)]

    for t in range(t_max):
        for w in range(w_max):
            L[w][t] = symbols(f"L[{w}][{t}]")
    
    for b in range(b_max):
        X[b] = symbols(f"X[{b}]")
    # ---------------- ---------------- ---------------- ----------------
    

    # Making 0 into False and 1 into True
    # Replacing x with X, so we use X[] and not x[] by accident
    # Replacing l with L, so we use L[][] and not l[][] by accident
    
    # Vector for the translation of v to the sympy symbols
    v_for_sympy = []

    # Translating from our version to SymPys
    i = 0
    while i < len(v):
        if v[i] == _false: v_for_sympy.append('False'); i+=1
        elif v[i] == _true: v_for_sympy.append('True'); i+=1
        elif v[i] == _and: v_for_sympy.append('&'); i+=1
        elif v[i] == _or: v_for_sympy.append('|'); i+=1
        else: 
            # turns all x to X and l to L
            v_i_translation = v[i].translate(str.maketrans({"x": "X", "l": "L"}))
            v_for_sympy.append(v_i_translation); i+=1


    # Putting all together in a string
    v_as_string = " ".join(v_for_sympy)

    # Creating actual SymPy expression
    expr = eval(v_as_string)

    # Simplify expression
    simplified_expr = simplify(simplify_logic(expr))

    # Convert back to the vector format
    simplified_v = convert_sympy_str_to_vector(str(simplified_expr))

    return simplified_v

In [1397]:
# Global Variables for program

v = [] # vector storing the ending expression

set_of_ks = {} # set that stores values like K(b,w,t,bv) and k(b,s_d,w,t)

v_of_vocab_sets = []

In [1398]:
def add_to_set_of_ks(base_k,v_prime):
    # adds to set_of_ks with base_k as key and v_prime as value
    if base_k not in set_of_ks:
        set_of_ks[base_k] = v_prime
        return True # Indicates that was added
    return False # Indicates that wasnt added (was already there)

In [1399]:
def iterate_B(B): # iterates through Bs terms, 
    # places values in set_of_ks,
    # TBD (use b_vocab) -> changes B to eliminate redundancy,
    # TBD () -> removes book b from B if no word to be learned (B[b] is an empty list)

    
    for b in range(len(B)):
        b_vocab = set()
        for s_d in range(len(B[b])):
            
            b_vocab.update(B[b][s_d])
            # Add all words in s_d to vocabulary

            if len(B[b][s_d]) == 1:

                w = B[b][s_d][0]
                base_K = f"K_reread({b},{w},0)"
                add_to_set_of_ks(base_K,[_true])
                # if s_d of book b has 1 word (w), 
                # we register that we can learn w by reading book b
        
        # Add set of book vocab to the global list
        v_of_vocab_sets.append(b_vocab)
                

In [1400]:
def print_vector(vec):
    for x in vec:
        print(x, end=" ")
    print()  # final newline


In [1401]:
def vector_x_eq_int(b, t):
    # x_eq_int = ["(",f"Eq(x[{b}],{t})",")"]
    x_eq_int = [f"Eq(x[{b}],{t})"]
    return x_eq_int

def andify(vector_of_ands): # vector_of_ands is a vector of vectors
    v = []
    and_symbol_string = _and
    
    for i in range(len(vector_of_ands)):
        v.append("(")
        if vector_of_ands[i] == []:
            # if empty its the same as AND 1
            v.append('1')
        else:
            v.extend(vector_of_ands[i])
        v.append(")")
        if i != len(vector_of_ands)-1: 
            # if not the last, append an "AND" after
            v.append(and_symbol_string)
    return v

def orify(vector_of_ors): # vector_of_ors is a vector of vectors
    v = []
    or_symbol_string = _or
    
    for i in range(len(vector_of_ors)):
        v.append("(")
        if vector_of_ors[i] == []: 
            # if empty its the same as OR 0
            v.append(_false)
        else:
            v.extend(vector_of_ors[i])
        v.append(")")
        if i != len(vector_of_ors)-1: 
            # if not the last, append an "OR" after
            v.append(or_symbol_string)
    return v


In [1402]:
def clean_constants(v_prime): # by ChatGPT # removes 0 OR or OR 0 and 1 AND or AND 1
    v = v_prime.copy()
    i = 0
    while i < len(v) - 2:
        a, op, b = v[i], v[i+1], v[i+2]

        # Case: 0 OR X → X
        if a == '0' and op == "OR":
            del v[i:i+2]   # remove 0, "OR"
            continue

        # Case: X OR 0 → X
        if op == "OR" and b == '0':
            del v[i+1:i+3]  # remove "OR", 0
            continue

        # Case: 1 AND X → X
        if a == '1' and op == "AND":
            del v[i:i+2]   # remove 1, "AND"
            continue

        # Case: X AND 1 → X
        if op == "AND" and b == '1':
            del v[i+1:i+3]  # remove "AND", 1
            continue

        i += 1
    return v

def collapse_parentheses(v_prime): # by Gemini # turns whole exp into 1 if OR 1 and into 0 if AND 0
    """
    Collapses parenthetical groups in a boolean vector if they match
    short-circuit conditions (AND 0, OR 1) at the top level of the group.
    
    Args:
        v_prime (list): A list of strings and integers (e.g., ['(', 'A', 'AND', 0, ')'])
    
    Returns:
        list: The modified list with short-circuits collapsed.
    """
    v = v_prime.copy()
    
    # We use a while loop because we modify the list length in place
    i = 0
    while i < len(v):
        if v[i] == "(":
            start = i
            
            # 1. Find the MATCHING closing parenthesis
            # We cannot just use .index(")") because that might catch a nested one.
            balance = 1
            end = -1
            for k in range(i + 1, len(v)):
                if v[k] == "(":
                    balance += 1
                elif v[k] == ")":
                    balance -= 1
                
                if balance == 0:
                    end = k
                    break
            
            # If no matching paren found (malformed), just move on
            if end == -1:
                i += 1
                continue

            # 2. Scan INSIDE the identified range [start+1 : end]
            # We look for short circuits, but we must SKIP nested parentheses
            j = start + 1
            found_short_circuit = None # Can be 0 or 1
            
            while j < end:
                token = v[j]
                
                # If we hit a nested open paren, skip this entire nested block
                if token == "(":
                    nested_balance = 1
                    while j < end and nested_balance > 0:
                        j += 1
                        if v[j] == "(": nested_balance += 1
                        elif v[j] == ")": nested_balance -= 1
                    # j is now at the closing paren of the nested group
                    j += 1
                    continue

                # CHECK FOR: AND 0
                # We check if token is 0, and if it has an AND neighbor
                if token == '0':
                    prev_is_and = (j > start + 1 and v[j-1] == "AND")
                    next_is_and = (j + 1 < end and v[j+1] == "AND")
                    
                    if prev_is_and or next_is_and:
                        found_short_circuit = '0'
                        break

                # CHECK FOR: OR 1
                # We check if token is 1, and if it has an OR neighbor
                if token == '1':
                    prev_is_or = (j > start + 1 and v[j-1] == "OR")
                    next_is_or = (j + 1 < end and v[j+1] == "OR")
                    
                    if prev_is_or or next_is_or:
                        found_short_circuit = '1'
                        break
                
                j += 1

            # 3. Apply Collapse if found
            if found_short_circuit is not None:
                # Replace everything from start to end (inclusive) with the result
                v[start:end+1] = [found_short_circuit]
                # Reset cursor to start to re-evaluate surrounding context if needed
                # (e.g., ( (A) AND 0 ) AND 0 might need two passes, checking from start is safest)
                i = 0 
            else:
                i += 1
        else:
            i += 1
            
    return v

def remove_redundant_nested_parentheses(v_prime): # by ChatGPT # turns ( ( A ) ) into ( A )
    v = v_prime.copy()
    """
    In-place: collapse only exact redundant nested parentheses of the form
      ( ( ... ) )
    into
      ( ... )

    Safety: only removes when the inner '(''s matching ')' is immediately
    followed by the outer ')'. This avoids removing '(' in cases like
      ( ( A OR B ) AND C )
    which must be preserved.
    """
    changed = True
    while changed:
        changed = False
        i = 0
        while i < len(v) - 1:
            # look for "( ("
            if v[i] == "(" and v[i+1] == "(":
                # find matching close for the inner '(' at position i+1
                depth = 0
                j = i + 1
                matched = -1
                while j < len(v):
                    if v[j] == "(":
                        depth += 1
                    elif v[j] == ")":
                        depth -= 1
                        if depth == 0:
                            matched = j
                            break
                    j += 1

                # matched is index of inner ')' if found
                # check that the token right after inner ')' is an outer ')'
                if matched != -1 and matched + 1 < len(v) and v[matched + 1] == ")":
                    # remove the outermost redundant pair:
                    # delete v[i] (the first '(') and v[matched + 1] (the corresponding outer ')')
                    # delete the later index first to avoid shifting problem
                    del v[matched + 1]
                    del v[i]
                    changed = True
                    # after deletion, stay at same i to re-check nested chains e.g. (((X))) 
                    continue
                else:
                    # not a simple redundant nested pair; advance
                    i += 1
            else:
                i += 1
    return v

def remove_redundant_nested_parentheses2(v_prime): # function that makes ( 0 ) into 0 and ( 1 ) into 1
    v = v_prime.copy()
    i = 0
    while i < len(v) - 2:
        if v[i] == "(" and v[i+2] == ")":
            if v[i+1] == '0' or v[i+1] == '1':
                # remove the parentheses
                v[i:i+3] = [v[i+1]]
                continue
        i += 1
    return v

def clean_andor_parentheses(v): # function that cleans things like AND ) and makes it just be )
    v_prime = v.copy()
    i = 0
    
    # Loop until the second to last element (since we check i+1)
    while i < len(v_prime) - 1:
        
        current_token = v_prime[i]
        next_token = v_prime[i+1]
        
        if current_token in ("AND", "OR") and next_token == ")":
            del v_prime[i]
            # Do not advance i; check the new token at this index

        elif current_token == "(" and next_token in ("AND", "OR"):
            del v_prime[i+1]
            # Do not advance i; check if another operator slid into place
            
        else:
            i += 1
            
    return v_prime
    

def clean_vector(v):
    v_prime = v.copy()
    
    while True: # repeat cleaning until no changes
        comparison_vector = v_prime.copy()
        
        v_prime = clean_andor_parentheses(v_prime)
        v_prime = remove_redundant_nested_parentheses2(v_prime)
        v_prime = clean_constants(v_prime)
        v_prime = collapse_parentheses(v_prime)
        v_prime = remove_redundant_nested_parentheses(v_prime)

        if comparison_vector == v_prime:
            break
    return v_prime


In [1403]:
def K_reread(b, w, t): # Can I learn word w from book b at time t?
    # This K is the older version, which allows rereads, 
    # but hopefully can provide more aggregation, leaving the reread to be dealt by another constraint
    
    # Check if we know K (we registered some values with t==0 in iterate_B)
    if t == 0:
        base_K = f"K_reread({b},{w},{t})"
        if base_K in set_of_ks:
            return set_of_ks[base_K]
    
    # check if book b has w
    if w not in v_of_vocab_sets[b]:
        # w is not to be found in book b

        if t == 0:
            # we read no books yet (t = 0) and 
            # book b doesnt have w in it

            # So, we cant learn it through book b, and return 0
            return [_false]
        
        # maybe we could have learned it in a past book
        # So, we check if we knew it at t-1 (l[w][t-1])

        # Check if they are True or False or has size 1 (is Eq(x[b],t)) 
        # so we can replace them already instead of a string of l[w][t-1]
        if ( l[w][t-1] == [_false] or l[w][t-1] == [_true] 
             or len(l[w][t-1]) == 1):
            return l[w][t-1]

        v_prime = [f"l[{w}][{t-1}]"]
        # v_prime = l[w][t-1]
        # ------ Change l ------

        return v_prime
    
        
    
    # At this point, we know:
    #   1. Word w is in b
    #   2. Word w is not alone in a sentence in b (else it would be in vocab)

    v_prime = []

    v_of_ors = []

    for s_d in range(len(B[b])):
        
        if w in B[b][s_d]:

            # This 'if' will never be entered because of "At this point, we know:" point 2
            if len(B[b][s_d]) == 1: 
                # w is alone in s_d, so we learned w in this book
                # So, we can return a 1, cause we will learn w in this book b
                return [_true]

            # if w is not alone, we need to know
            # all other words omega
            v_of_ands = []
            for omega in B[b][s_d]:

                if omega == w:
                    continue
                
                v_of_ands.append(k(b, s_d, omega, t))
            v_of_ands = andify(v_of_ands) 
            # Make the AND of all ks of the words in s_d

            v_of_ors.append(v_of_ands)
    v_prime = orify(v_of_ors)
    # Make the OR of knowing w in all sentences in book b

    return v_prime


In [1404]:
def k(b, s_d, omega, t):
    # return [f"k({b},{s_d},{omega},{t})"]
    base_k = f"k({b},{s_d},{omega},{t})"

    # Check if k(b,s_d,omega,t) was already calculated 
    # before and return its value if it was
    if base_k in set_of_ks:
        return set_of_ks[base_k]

    
    if len(B[b][s_d]) == 1 and B[b][s_d][0] == omega:
        # omega is the only word in sentence s_d of book b

        # Add before returning
        add_to_set_of_ks(base_k,[_true])

        # So, we learn omega reading book b at s_d. 
        # Hence, returning 1 for value of k
        return [_true]
    

    
    if s_d == 0: # starting sentence
        if t == 0: # starting book
            # if we are at the starting s_d and starting book
            # and s_d has more than 1 word, we cant learn it
            # so we return [0]

            # Add before returning
            add_to_set_of_ks(base_k,[_false])

            return [_false]
        
        
        # if first sentence, we can only look back at t-1
        
        # Check if they are True or False or has size 1 (is Eq(x[b*],t*) or l[w*][t*])
        # so we can replace them already instead of a string of l[omega][t-1]
        if ( l[omega][t-1] == [_false] or l[omega][t-1] == [_true] 
             or len(l[omega][t-1]) == 1):
            # if l[omega][t-1] is a single value, 
            # we can return it as the value to simplify later

            # Add before returning
            add_to_set_of_ks(base_k,l[omega][t-1])
            
            return l[omega][t-1]

        # Add before returning
        add_to_set_of_ks(base_k,[f"l[{omega}][{t-1}]"])

        # return l[omega][t-1]
        return [f"l[{omega}][{t-1}]"]
        # --- Change l ---
    
    v_prime = []

    v_of_ors = []

    # EITHER we knew omega in the last sentence (s_d-1)
    v_of_ors.append(k(b, s_d-1, omega, t))

    # OR if (omega is in s_d) we know ALL the other words w 
    # in the same s_d of book b ( make an AND of k(b,s_d-1,w,t) )
    # If (omega not in s_d) we can just rely on knowing it in s_d-1, 
    # which is already in v_of_ors
    if omega in B[b][s_d]:
        v_of_ands = []

        for w in B[b][s_d]:
            if w == omega:
                continue

            v_of_ands.append(k(b, s_d-1, w, t))

        v_of_ands = andify(v_of_ands)
        v_of_ors.append(v_of_ands)
    
    v_prime = orify(v_of_ors)

    # Add before returning
    add_to_set_of_ks(base_k,v_prime)

    return v_prime

In [1405]:
# Books
#  Creating the Modelling


# B = [ 
#       [ [1] ] ,                          # b0
#       [ [0,1] ] ,                        # b1
#       [ [0,3], [2], [4,3,5], [4,0] ] ,   # b2
#       [ [0,3], [2], [4,3,5], [4,0] ] ,   # b3
#     ]

B = [ # good example for testing rereads
      [ [1,2,3,4],[1,2,3],[1,2],[1] ] ,                          # b0
      [ [5] ] ,                        # b1
      [ [0] ] ,   # b2
      [ [5], [1,2,0] ] ,   # b3
    ]

iterate_B(B) # puts vaues in set_of_ks

n_of_unknown_words = 6 #max number in B - 1

n_of_books_to_read = 4 # At most the amount of books; max value t+1 can have.

n_of_books = len(B)
# L = 

In [1406]:
# See later (full_build())
    # def full_build():
    #     for b in range(len(B)):
    #         for s_d in range(len(B[b])):
    #             for w in B[b][s_d]:
    #                 for t in range(n_of_books_to_read):

    #                     L[][][]    
                        
                    
                    


    # L = [w][t][b] of 
    #   n° of words BY
    #   n° of books to read (can be less than amount of books)) BY
    #   n° of books

    # L = [[[[] for b in range(len(B))] for t in range(n_of_books_to_read)] for w in range(n_of_unknown_words)]

    # inside L[w][t][b] there will be K(b,w,t)
    # in each l[w][t] we will do OR between all b

In [1407]:
l = [[[] for t in range(n_of_books_to_read)] for w in range(n_of_unknown_words)]

total_time = 0.0
for t in range(n_of_books_to_read):
    # verify run time
    vector_time_t = []
    for w in range(n_of_unknown_words):
        
        v_of_ors1 = []

        # Create "l[w][t-1] OR"
        if t != 0:
            if l[w][t-1] != [_false]:
                
                # Check if l[w][t-1] == [1], 
                # if yes we can say that any l[w][t*>t-1] == 1
                # So we just return [1]
                if l[w][t-1] == [_true]:
                    l[w][t] = [_true]
                    continue

                # Check if has size 1 (is Eq(x[b*],t*) or l[w*][t*])
                if (len(l[w][t-1]) == 1):
                    v_of_ors1.append(l[w][t-1])

                # If not, just add string version
                else:    
                    v_of_ors1.append([f"l[{w}][{t-1}]"])
                    # v_of_ors1.append(l[w][t-1])
                    # --- Change l ---

        
        v_prime = []
        v_of_ors2 = []

        for b in range(n_of_books):

            v_of_ands = []

            # Create "(x[b] == t) AND"
            v_of_ands.append(vector_x_eq_int(b,t))
            # Create "K_reread(b,w,t)""
            v_of_ands.append(K_reread(b,w,t))

            v_of_ands = andify(v_of_ands)
            v_of_ors2.append(v_of_ands)
        
        v_of_ors2 = orify(v_of_ors2)
        v_of_ors1.append(v_of_ors2)

        v_prime = orify(v_of_ors1)

        l[w][t] = v_prime

        start = time.perf_counter() # verify run time
        l[w][t] = clean_vector_sympy(l[w][t],t,n_of_unknown_words,n_of_books)
        end = time.perf_counter() # verify run time

        # verify run time
        elapsed = end - start
        total_time += elapsed
        vector_time_t.append(f"{elapsed:.5f}")
    print(f"l[*][{t}] time = {vector_time_t}") # verify run time
print(f"Total time: {round(total_time,5)}") # verify run time


l[*][0] time = ['0.00144', '0.00109', '0.00047', '0.00037', '0.00034', '0.00479']
l[*][1] time = ['0.04212', '0.04086', '0.06128', '0.00069', '0.00047', '0.04188']
l[*][2] time = ['0.10211', '0.09477', '0.08100', '0.01761', '0.00065', '0.04383']
l[*][3] time = ['0.09419', '0.09151', '0.25212', '0.06179', '0.02016', '0.04473']
Total time: 1.10027


In [1408]:
# Results Display
for t in range(n_of_books_to_read): # n_of_books_to_read
    for w in range(n_of_unknown_words): # n_of_unknown_words
        print(f"l[{w}][{t}]:")
        print("     ",end="")
        print_vector(l[w][t])
        
        print()

# print_vector(l[3][1])

l[0][0]:
     Eq(x[2],0) 

l[1][0]:
     Eq(x[0],0) 

l[2][0]:
     0 

l[3][0]:
     0 

l[4][0]:
     0 

l[5][0]:
     Eq(x[1],0) OR Eq(x[3],0) 

l[0][1]:
     Eq(x[2],0) OR Eq(x[2],1) 

l[1][1]:
     Eq(x[0],0) OR Eq(x[0],1) 

l[2][1]:
     Eq(x[0],0) AND (Eq(x[0],1) OR Eq(x[2],0)) AND (Eq(x[0],1) OR Eq(x[3],1)) 

l[3][1]:
     0 

l[4][1]:
     0 

l[5][1]:
     l[5][0] OR Eq(x[1],1) OR Eq(x[3],1) 

l[0][2]:
     l[0][1] OR Eq(x[2],2) OR (l[1][1] AND l[2][1] AND Eq(x[3],2)) 

l[1][2]:
     l[1][1] OR Eq(x[0],2) OR (l[0][1] AND l[2][1] AND Eq(x[3],2)) 

l[2][2]:
     l[2][1] OR (l[1][1] AND Eq(x[0],2)) OR (l[0][1] AND l[1][1] AND Eq(x[3],2)) 

l[3][2]:
     l[1][1] AND l[2][1] AND Eq(x[0],2) 

l[4][2]:
     0 

l[5][2]:
     l[5][1] OR Eq(x[1],2) OR Eq(x[3],2) 

l[0][3]:
     l[0][2] OR Eq(x[2],3) OR (l[1][2] AND l[2][2] AND Eq(x[3],3)) 

l[1][3]:
     l[1][2] OR Eq(x[0],3) OR (l[0][2] AND l[2][2] AND Eq(x[3],3)) 

l[2][3]:
     l[2][2] OR (l[1][2] AND Eq(x[0],3)) OR (l[0][2] AND l

In [1409]:
print(l[4][1])

['0']


In [ ]:
# Rascunho em Python 5

l[*][0] time = ['0.00147', '0.00124', '0.00053', '0.00042', '0.00046', '0.00526']
l[*][1] time = ['0.21801', '0.22605', '0.05498', '0.00146', '0.00071', '0.04474']
l[*][2] time = ['0.10026', '0.09758', '0.12758', '0.39025', '0.08802', '0.04537']
l[*][3] time = ['0.09954', '0.09630', '0.11989', '0.17735', '0.16393', '0.04442']

Total time: 2.10584

# -----------------------------------------------------------------------

l[*][0] time = ['0.00144', '0.00109', '0.00047', '0.00037', '0.00034', '0.00479']
l[*][1] time = ['0.04212', '0.04086', '0.06128', '0.00069', '0.00047', '0.04188']
l[*][2] time = ['0.10211', '0.09477', '0.08100', '0.01761', '0.00065', '0.04383']
l[*][3] time = ['0.09419', '0.09151', '0.25212', '0.06179', '0.02016', '0.04473']

Total time: 1.10027

SyntaxError: invalid syntax (1386608282.py, line 3)

In [ ]:
# Rascunho em Python 4

l[0][0]:
     Eq(x[2],0) 

l[1][0]:
     Eq(x[0],0) 

l[2][0]:
     0 

l[3][0]:
     0 

l[4][0]:
     0 

l[5][0]:
     Eq(x[1],0) OR Eq(x[3],0) 

l[0][1]:
     Eq(x[2],0) OR Eq(x[2],1) 

l[1][1]:
     Eq(x[0],0) OR Eq(x[0],1) 

l[2][1]:
     Eq(x[0],0) AND 
     (Eq(x[0],1) OR Eq(x[2],0)) AND 
     (Eq(x[0],1) OR Eq(x[3],1)) 

l[3][1]:
     0 

l[4][1]:
     0 

l[5][1]:
     l[5][0] OR 
     Eq(x[1],1) OR 
     Eq(x[3],1) 

l[0][2]:
     l[0][1] OR 
     Eq(x[2],2) OR 
     (l[1][1] AND l[2][1] AND Eq(x[3],2)) 

l[1][2]:
     l[1][1] OR 
     Eq(x[0],2) OR 
     (l[0][1] AND l[2][1] AND Eq(x[3],2)) 

l[2][2]:
     l[2][1] OR 
     (l[1][1] AND Eq(x[0],2)) OR 
     (l[0][1] AND l[1][1] AND Eq(x[3],2)) 

l[3][2]:
     l[1][1] AND 
     l[2][1] AND 
     Eq(x[0],2) 

l[4][2]:
     0 

l[5][2]:
     l[5][1] OR 
     Eq(x[1],2) OR 
     Eq(x[3],2) 

l[0][3]:
     l[0][2] OR 
     Eq(x[2],3) OR 
     (l[1][2] AND l[2][2] AND Eq(x[3],3)) 

l[1][3]:
     l[1][2] OR 
     Eq(x[0],3) OR 
     (l[0][2] AND l[2][2] AND Eq(x[3],3)) 

l[2][3]:
     l[2][2] OR 
     (l[1][2] AND Eq(x[0],3)) OR 
     (l[0][2] AND l[1][2] AND Eq(x[3],3)) 

l[3][3]:
     l[3][2] OR 
     (l[1][2] AND l[2][2] AND Eq(x[0],3)) 

l[4][3]:
     l[1][2] AND 
     l[2][2] AND 
     l[3][2] AND 
     Eq(x[0],3) 

l[5][3]:
     l[5][2] OR 
     Eq(x[1],3) OR 
     Eq(x[3],3)

SyntaxError: invalid syntax (1110901855.py, line 3)

In [ ]:
# Rascunho em Python 3

l[2][1]: # K not expanded
     ( 
          ( l[2][0] ) OR 
          
          ( 
               ( x[0] == 1 ) AND 
               ( ( ( x[1] == 0 ) AND ( K(0,2,1,[1]) ) ) OR ( ( x[2] == 0 ) AND ( K(0,2,1,[2]) ) ) OR ( ( x[3] == 0 ) AND ( K(0,2,1,[3]) ) ) ) 
          ) OR 
          
          ( 
               ( x[1] == 1 ) AND 
               ( ( ( x[0] == 0 ) AND ( K(1,2,1,[0]) ) ) OR ( ( x[2] == 0 ) AND ( K(1,2,1,[2]) ) ) OR ( ( x[3] == 0 ) AND ( K(1,2,1,[3]) ) ) ) 
          ) OR 
          
          ( 
               ( x[2] == 1 ) AND 
               ( ( ( x[0] == 0 ) AND ( K(2,2,1,[0]) ) ) OR ( ( x[1] == 0 ) AND ( K(2,2,1,[1]) ) ) OR ( ( x[3] == 0 ) AND ( K(2,2,1,[3]) ) ) ) 
          ) OR 
          
          ( 
               ( x[3] == 1 ) AND 
               ( ( ( x[0] == 0 ) AND ( K(3,2,1,[0]) ) ) OR ( ( x[1] == 0 ) AND ( K(3,2,1,[1]) ) ) OR ( ( x[2] == 0 ) AND ( K(3,2,1,[2]) ) ) ) 
          ) 
     )



l[2][1]: # k not expanded
     ( 
          ( l[2][0] ) OR 
          
          ( 
               ( x[0] == 1 ) AND 
               
               ( 
                    ( ( x[1] == 0 ) AND ( ( k(0,0,1,1) AND k(0,0,3,1) AND k(0,0,4,1) ) OR ( k(0,1,1,1) AND k(0,1,3,1) ) OR ( k(0,2,1,1) ) ) ) OR 
                    
                    ( ( x[2] == 0 ) AND ( ( k(0,0,1,1) AND k(0,0,3,1) AND k(0,0,4,1) ) OR ( k(0,1,1,1) AND k(0,1,3,1) ) OR ( k(0,2,1,1) ) ) ) OR 
                    
                    ( ( x[3] == 0 ) AND ( ( k(0,0,1,1) AND k(0,0,3,1) AND k(0,0,4,1) ) OR ( k(0,1,1,1) AND k(0,1,3,1) ) OR ( k(0,2,1,1) ) ) ) 
               ) 
          )
     )

l[2][1]:
     ( 
          ( x[0] == 1 ) AND 

          ( 
               ( ( x[1] == 0 ) AND ( l[1][0] ) ) OR 
               ( ( x[2] == 0 ) AND ( l[1][0] ) ) OR 
               ( ( x[3] == 0 ) AND ( l[1][0] ) ) 
          ) 
     )

l[2][1] ( x[0] == 1 ) AND ( x[1] == 0 ) AND ( l[1][0] )


l[1][0]:
     ( x[0] == 0 )

l[1][1]:
     ( 
          ( x[0] == 0 ) OR 
          ( ( x[0] == 1 ) AND ( ( x[1] == 0 ) OR ( x[2] == 0 ) OR ( x[3] == 0 ) ) ) OR 
          ( ( x[1] == 1 ) AND ( x[0] == 0 ) ) OR 
          ( ( x[2] == 1 ) AND ( x[0] == 0 ) ) OR 
          ( ( x[3] == 1 ) AND ( x[0] == 0 ) ) 
     )

l[1][1] 
     A or (B and (C or D or F)) or (G and A) or (H and A) or (I and A)
     A or (B and (C or D or F)) or ((G or H or I) and A)


l[2][2]:
     ( 
          ( l[2][1] ) OR 
          
          ( 
               ( x[0] == 2 ) AND 
               
               ( 
                    ( 
                         ( x[1] == 1 ) AND 
                         ( 
                              ( 
                                   ( x[2] == 0 ) AND 
                                   ( ( ( l[1][1] ) AND ( l[1][1] AND l[2][1] ) ) OR ( ( l[1][1] ) OR ( l[2][1] ) ) ) 
                              ) OR 

                              ( 
                                   ( x[3] == 0 ) AND 
                                   ( ( ( l[1][1] ) AND ( l[1][1] AND l[2][1] ) ) OR ( ( l[1][1] ) OR ( l[2][1] ) ) ) 
                              ) 
                         )
                    ) OR 
                    
                    ( 
                         ( x[2] == 1 ) AND 
                         ( 
                              ( ( x[1] == 0 ) AND ( ( ( l[1][1] ) AND ( l[1][1] AND l[2][1] ) ) OR ( ( l[1][1] ) OR ( l[2][1] ) ) ) ) OR 
                              
                              ( ( x[3] == 0 ) AND ( ( ( l[1][1] ) AND ( l[1][1] AND l[2][1] ) ) OR ( ( l[1][1] ) OR ( l[2][1] ) ) ) ) 
                         ) 
                    ) OR 
                    
                    ( 
                         ( x[3] == 1 ) AND 
                         ( 
                              ( ( x[1] == 0 ) AND ( ( ( l[1][1] ) AND ( l[1][1] AND l[2][1] ) ) OR ( ( l[1][1] ) OR ( l[2][1] ) ) ) ) OR 
                              
                              ( ( x[2] == 0 ) AND ( ( ( l[1][1] ) AND ( l[1][1] AND l[2][1] ) ) OR ( ( l[1][1] ) OR ( l[2][1] ) ) ) ) 
                         ) 
                    ) 
               ) 
          )
     )



SyntaxError: invalid syntax (3325965032.py, line 3)

In [ ]:
# Rascunho em Python 2

l[2][0]:
     ( ( ( x[0] == 0 ) AND ( K(0,2,0,[]) ) ) OR ( ( x[1] == 0 ) AND ( K(1,2,0,[]) ) ) OR ( ( x[2] == 0 ) AND ( K(2,2,0,[]) ) ) OR ( ( x[3] == 0 ) AND ( K(3,2,0,[]) ) ) ) 

l[2][1]:
     ( ( l[2][0] ) OR ( ( x[0] == 1 ) AND ( ( ( x[1] == 0 ) AND ( K(0,2,1,[1]) ) ) OR ( ( x[2] == 0 ) AND ( K(0,2,1,[2]) ) ) OR ( ( x[3] == 0 ) AND ( K(0,2,1,[3]) ) ) ) ) OR ( ( x[1] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(1,2,1,[0]) ) ) OR ( ( x[2] == 0 ) AND ( K(1,2,1,[2]) ) ) OR ( ( x[3] == 0 ) AND ( K(1,2,1,[3]) ) ) ) ) OR ( ( x[2] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(2,2,1,[0]) ) ) OR ( ( x[1] == 0 ) AND ( K(2,2,1,[1]) ) ) OR ( ( x[3] == 0 ) AND ( K(2,2,1,[3]) ) ) ) ) OR ( ( x[3] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(3,2,1,[0]) ) ) OR ( ( x[1] == 0 ) AND ( K(3,2,1,[1]) ) ) OR ( ( x[2] == 0 ) AND ( K(3,2,1,[2]) ) ) ) ) ) 

l[2][2]:
     ( 
          ( l[2][1] ) OR 
          
          ( ( x[0] == 2 ) AND 
               ( 
                    ( ( x[1] == 1 ) AND 
                         ( 
                              ( ( x[2] == 0 ) AND ( K(0,2,2,[2, 1]) ) ) OR 
                              ( ( x[3] == 0 ) AND ( K(0,2,2,[3, 1]) ) ) ) 
                    ) 
                         
                    OR 

                    ( ( x[2] == 1 ) AND ( ( ( x[1] == 0 ) AND ( K(0,2,2,[1, 2]) ) ) OR ( ( x[3] == 0 ) AND ( K(0,2,2,[3, 2]) ) ) ) ) OR 
                    
                    ( ( x[3] == 1 ) AND ( ( ( x[1] == 0 ) AND ( K(0,2,2,[1, 3]) ) ) OR ( ( x[2] == 0 ) AND ( K(0,2,2,[2, 3]) ) ) ) ) ) 
          ) 
                    
          OR 
          
          ( ( x[1] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( ( x[2] == 0 ) AND ( K(1,2,2,[2, 0]) ) ) OR ( ( x[3] == 0 ) AND ( K(1,2,2,[3, 0]) ) ) ) ) OR ( ( x[2] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(1,2,2,[0, 2]) ) ) OR ( ( x[3] == 0 ) AND ( K(1,2,2,[3, 2]) ) ) ) ) OR ( ( x[3] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(1,2,2,[0, 3]) ) ) OR ( ( x[2] == 0 ) AND ( K(1,2,2,[2, 3]) ) ) ) ) ) ) OR 
          
          ( ( x[2] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( ( x[1] == 0 ) AND ( K(2,2,2,[1, 0]) ) ) OR ( ( x[3] == 0 ) AND ( K(2,2,2,[3, 0]) ) ) ) ) OR ( ( x[1] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(2,2,2,[0, 1]) ) ) OR ( ( x[3] == 0 ) AND ( K(2,2,2,[3, 1]) ) ) ) ) OR ( ( x[3] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(2,2,2,[0, 3]) ) ) OR ( ( x[1] == 0 ) AND ( K(2,2,2,[1, 3]) ) ) ) ) ) ) OR 
          
          ( ( x[3] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( ( x[1] == 0 ) AND ( K(3,2,2,[1, 0]) ) ) OR ( ( x[2] == 0 ) AND ( K(3,2,2,[2, 0]) ) ) ) ) OR ( ( x[1] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(3,2,2,[0, 1]) ) ) OR ( ( x[2] == 0 ) AND ( K(3,2,2,[2, 1]) ) ) ) ) OR ( ( x[2] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(3,2,2,[0, 2]) ) ) OR ( ( x[1] == 0 ) AND ( K(3,2,2,[1, 2]) ) ) ) ) ) ) 
     ) 

l[2][3]:
     ( ( l[2][2] ) OR ( ( x[0] == 3 ) AND ( ( ( x[1] == 2 ) AND ( ( ( x[2] == 1 ) AND ( ( x[3] == 0 ) AND ( K(0,2,3,[3, 2, 1]) ) ) ) OR ( ( x[3] == 1 ) AND ( ( x[2] == 0 ) AND ( K(0,2,3,[2, 3, 1]) ) ) ) ) ) OR ( ( x[2] == 2 ) AND ( ( ( x[1] == 1 ) AND ( ( x[3] == 0 ) AND ( K(0,2,3,[3, 1, 2]) ) ) ) OR ( ( x[3] == 1 ) AND ( ( x[1] == 0 ) AND ( K(0,2,3,[1, 3, 2]) ) ) ) ) ) OR ( ( x[3] == 2 ) AND ( ( ( x[1] == 1 ) AND ( ( x[2] == 0 ) AND ( K(0,2,3,[2, 1, 3]) ) ) ) OR ( ( x[2] == 1 ) AND ( ( x[1] == 0 ) AND ( K(0,2,3,[1, 2, 3]) ) ) ) ) ) ) ) OR ( ( x[1] == 3 ) AND ( ( ( x[0] == 2 ) AND ( ( ( x[2] == 1 ) AND ( ( x[3] == 0 ) AND ( K(1,2,3,[3, 2, 0]) ) ) ) OR ( ( x[3] == 1 ) AND ( ( x[2] == 0 ) AND ( K(1,2,3,[2, 3, 0]) ) ) ) ) ) OR ( ( x[2] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( x[3] == 0 ) AND ( K(1,2,3,[3, 0, 2]) ) ) ) OR ( ( x[3] == 1 ) AND ( ( x[0] == 0 ) AND ( K(1,2,3,[0, 3, 2]) ) ) ) ) ) OR ( ( x[3] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( x[2] == 0 ) AND ( K(1,2,3,[2, 0, 3]) ) ) ) OR ( ( x[2] == 1 ) AND ( ( x[0] == 0 ) AND ( K(1,2,3,[0, 2, 3]) ) ) ) ) ) ) ) OR ( ( x[2] == 3 ) AND ( ( ( x[0] == 2 ) AND ( ( ( x[1] == 1 ) AND ( ( x[3] == 0 ) AND ( K(2,2,3,[3, 1, 0]) ) ) ) OR ( ( x[3] == 1 ) AND ( ( x[1] == 0 ) AND ( K(2,2,3,[1, 3, 0]) ) ) ) ) ) OR ( ( x[1] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( x[3] == 0 ) AND ( K(2,2,3,[3, 0, 1]) ) ) ) OR ( ( x[3] == 1 ) AND ( ( x[0] == 0 ) AND ( K(2,2,3,[0, 3, 1]) ) ) ) ) ) OR ( ( x[3] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( x[1] == 0 ) AND ( K(2,2,3,[1, 0, 3]) ) ) ) OR ( ( x[1] == 1 ) AND ( ( x[0] == 0 ) AND ( K(2,2,3,[0, 1, 3]) ) ) ) ) ) ) ) OR ( ( x[3] == 3 ) AND ( ( ( x[0] == 2 ) AND ( ( ( x[1] == 1 ) AND ( ( x[2] == 0 ) AND ( K(3,2,3,[2, 1, 0]) ) ) ) OR ( ( x[2] == 1 ) AND ( ( x[1] == 0 ) AND ( K(3,2,3,[1, 2, 0]) ) ) ) ) ) OR ( ( x[1] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( x[2] == 0 ) AND ( K(3,2,3,[2, 0, 1]) ) ) ) OR ( ( x[2] == 1 ) AND ( ( x[0] == 0 ) AND ( K(3,2,3,[0, 2, 1]) ) ) ) ) ) OR ( ( x[2] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( x[1] == 0 ) AND ( K(3,2,3,[1, 0, 2]) ) ) ) OR ( ( x[1] == 1 ) AND ( ( x[0] == 0 ) AND ( K(3,2,3,[0, 1, 2]) ) ) ) ) ) ) ) )

SyntaxError: invalid syntax (187261274.py, line 3)

In [ ]:
# Rascunho em Python 1
( # Rewrite 3
     
     # l[0][1]: 
     #      ( 
     #           ( (x[0] == 1) AND 
     #                ( (x[0] == 0) AND (K(0,0,1,[0]) OR K(0,0,0,[])) ) OR
     #                ( (x[1] == 0) AND (K(0,0,1,[1]) OR K(1,0,0,[])) ) OR
     #                ( (x[2] == 0) AND (K(0,0,1,[2]) OR K(2,0,0,[])) ) OR
     #                ( (x[3] == 0) AND (K(0,0,1,[3]) OR K(3,0,0,[])) ) )
               
     #           # ( (x[0] == 1) AND 
     #           #      ( ((x[0] == 0) AND K(0,0,1,[0])) OR ((x[0] == 0) AND K(0,0,0,[])) ) OR
     #           #      ( ((x[1] == 0) AND K(0,0,1,[1])) OR ((x[1] == 0) AND K(1,0,0,[])) ) OR
     #           #      ( ((x[2] == 0) AND K(0,0,1,[2])) OR ((x[2] == 0) AND K(2,0,0,[])) ) OR
     #           #      ( ((x[3] == 0) AND K(0,0,1,[3])) OR ((x[3] == 0) AND K(3,0,0,[])) ) )

               
     #           OR
               
     #           ( (x[1] == 1) AND (x[0] == 0) AND (K(0,0,0,[]) OR K(1,0,1,[0])) ) OR
     #           ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,0,[]) OR K(1,0,1,[1])) ) OR
     #           ( (x[1] == 1) AND (x[2] == 0) AND (K(2,0,0,[]) OR K(1,0,1,[2])) ) OR
     #           ( (x[1] == 1) AND (x[3] == 0) AND (K(3,0,0,[]) OR K(1,0,1,[3])) )
               
     #           OR



     #           # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     #      ) 
)

( # Rewrite 4
# l[0][1]: 
#      ( 
#           ( (x[1] == 1) AND (x[0] == 0) AND (K(1,0,1,[0])) ) OR
#           ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,1,[1])) ) OR
#           ( (x[1] == 1) AND (x[2] == 0) AND (K(1,0,1,[2])) ) OR
#           ( (x[1] == 1) AND (x[3] == 0) AND (K(1,0,1,[3])) )

#           # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
#      ) 
)

( # Rewrite 5
# l[0][1]: 
#      ( 

#           ( (x[0] == 1) AND 
#                ( (x[0] == 0) AND (K(0,0,1,[0])) ) OR
#                ( (x[1] == 0) AND (K(0,0,1,[1])) ) OR
#                ( (x[2] == 0) AND (K(0,0,1,[2])) ) OR
#                ( (x[3] == 0) AND (K(0,0,1,[3])) ) )
          
#           OR

#           ( (x[1] == 1) AND 
#                ( (x[0] == 0) AND (K(1,0,1,[0])) ) OR
#                ( (x[1] == 0) AND (K(1,0,1,[1])) ) OR
#                ( (x[2] == 0) AND (K(1,0,1,[2])) ) OR
#                ( (x[3] == 0) AND (K(1,0,1,[3])) ) )

#           OR

#           ( (x[1] == 3) AND 
#                ( (x[0] == 2) AND (x[3] == 1) AND (x[2] == 0) AND (K_creator(2,0,[3,0,1])) ) OR
#                K(1,0,3,[2,3,0])


#                ( (x[1] == 0) AND (K(1,0,1,[1])) ) OR
#                ( (x[2] == 0) AND (K(1,0,1,[2])) ) OR
#                ( (x[3] == 0) AND (K(1,0,1,[3])) ) )

#           # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
#      ) 
)

l[0][2]: # actual output, not cleaned
     ( 
          ( ( x[0] == 2 ) AND 
               ( ( x[0] == 1 ) AND 0 ) OR 
               
               ( ( x[1] == 1 ) AND 
                    ( ( x[0] == 0 ) AND 0 ) OR 
                    ( ( x[1] == 0 ) AND 0 ) OR 
                    ( ( x[2] == 0 ) AND K(0,0,2,[2, 1]) ) OR 
                    ( ( x[3] == 0 ) AND K(0,0,2,[3, 1]) ) ) 
               
               OR 
               
               ( ( x[2] == 1 ) AND ( ( x[0] == 0 ) AND 0 ) OR ( ( x[1] == 0 ) AND K(0,0,2,[1, 2]) ) OR ( ( x[2] == 0 ) AND 0 ) OR ( ( x[3] == 0 ) AND K(0,0,2,[3, 2]) ) ) OR 
               
               ( ( x[3] == 1 ) AND ( ( x[0] == 0 ) AND 0 ) OR ( ( x[1] == 0 ) AND K(0,0,2,[1, 3]) ) OR ( ( x[2] == 0 ) AND K(0,0,2,[2, 3]) ) OR ( ( x[3] == 0 ) AND 0 ) ) ) 
               
          OR 
          
          ( ( x[1] == 2 ) AND ( ( x[0] == 1 ) AND ( ( x[0] == 0 ) AND 0 ) OR ( ( x[1] == 0 ) AND 0 ) OR ( ( x[2] == 0 ) AND K(1,0,2,[2, 0]) ) OR ( ( x[3] == 0 ) AND K(1,0,2,[3, 0]) ) ) OR ( ( x[1] == 1 ) AND 0 ) OR ( ( x[2] == 1 ) AND ( ( x[0] == 0 ) AND K(1,0,2,[0, 2]) ) OR ( ( x[1] == 0 ) AND 0 ) OR ( ( x[2] == 0 ) AND 0 ) OR ( ( x[3] == 0 ) AND K(1,0,2,[3, 2]) ) ) OR ( ( x[3] == 1 ) AND ( ( x[0] == 0 ) AND K(1,0,2,[0, 3]) ) OR ( ( x[1] == 0 ) AND 0 ) OR ( ( x[2] == 0 ) AND K(1,0,2,[2, 3]) ) OR ( ( x[3] == 0 ) AND 0 ) ) ) OR ( ( x[2] == 2 ) AND ( ( x[0] == 1 ) AND ( ( x[0] == 0 ) AND 0 ) OR ( ( x[1] == 0 ) AND K(2,0,2,[1, 0]) ) OR ( ( x[2] == 0 ) AND 0 ) OR ( ( x[3] == 0 ) AND K(2,0,2,[3, 0]) ) ) OR ( ( x[1] == 1 ) AND ( ( x[0] == 0 ) AND K(2,0,2,[0, 1]) ) OR ( ( x[1] == 0 ) AND 0 ) OR ( ( x[2] == 0 ) AND 0 ) OR ( ( x[3] == 0 ) AND K(2,0,2,[3, 1]) ) ) OR ( ( x[2] == 1 ) AND 0 ) OR ( ( x[3] == 1 ) AND ( ( x[0] == 0 ) AND K(2,0,2,[0, 3]) ) OR ( ( x[1] == 0 ) AND K(2,0,2,[1, 3]) ) OR ( ( x[2] == 0 ) AND 0 ) OR ( ( x[3] == 0 ) AND 0 ) ) ) OR ( ( x[3] == 2 ) AND ( ( x[0] == 1 ) AND ( ( x[0] == 0 ) AND 0 ) OR ( ( x[1] == 0 ) AND K(3,0,2,[1, 0]) ) OR ( ( x[2] == 0 ) AND K(3,0,2,[2, 0]) ) OR ( ( x[3] == 0 ) AND 0 ) ) OR ( ( x[1] == 1 ) AND ( ( x[0] == 0 ) AND K(3,0,2,[0, 1]) ) OR ( ( x[1] == 0 ) AND 0 ) OR ( ( x[2] == 0 ) AND K(3,0,2,[2, 1]) ) OR ( ( x[3] == 0 ) AND 0 ) ) OR ( ( x[2] == 1 ) AND ( ( x[0] == 0 ) AND K(3,0,2,[0, 2]) ) OR ( ( x[1] == 0 ) AND K(3,0,2,[1, 2]) ) OR ( ( x[2] == 0 ) AND 0 ) OR ( ( x[3] == 0 ) AND 0 ) ) OR ( ( x[3] == 1 ) AND 0 ) ) 
     )

l[0][2]:
     ( 
          ( ( x[0] == 2 ) AND 
               ( ( ( x[1] == 1 ) AND ( ( ( x[2] == 0 ) AND ( K(0,0,2,[2, 1]) ) ) OR ( ( x[3] == 0 ) AND ( K(0,0,2,[3, 1]) ) ) ) ) OR 
                
                ( ( x[2] == 1 ) AND ( ( ( x[1] == 0 ) AND ( K(0,0,2,[1, 2]) ) ) OR ( ( x[3] == 0 ) AND ( K(0,0,2,[3, 2]) ) ) ) ) OR 

                ( ( x[3] == 1 ) AND ( ( ( x[1] == 0 ) AND ( K(0,0,2,[1, 3]) ) ) OR ( ( x[2] == 0 ) AND ( K(0,0,2,[2, 3]) ) ) ) ) ) 
          ) 
               
          OR 

          ( ( x[1] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( ( x[2] == 0 ) AND ( K(1,0,2,[2, 0]) ) ) OR ( ( x[3] == 0 ) AND ( K(1,0,2,[3, 0]) ) ) ) ) OR ( ( x[2] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(1,0,2,[0, 2]) ) ) OR ( ( x[3] == 0 ) AND ( K(1,0,2,[3, 2]) ) ) ) ) OR ( ( x[3] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(1,0,2,[0, 3]) ) ) OR ( ( x[2] == 0 ) AND ( K(1,0,2,[2, 3]) ) ) ) ) ) ) OR 

          ( ( x[2] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( ( x[1] == 0 ) AND ( K(2,0,2,[1, 0]) ) ) OR ( ( x[3] == 0 ) AND ( K(2,0,2,[3, 0]) ) ) ) ) OR ( ( x[1] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(2,0,2,[0, 1]) ) ) OR ( ( x[3] == 0 ) AND ( K(2,0,2,[3, 1]) ) ) ) ) OR ( ( x[3] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(2,0,2,[0, 3]) ) ) OR ( ( x[1] == 0 ) AND ( K(2,0,2,[1, 3]) ) ) ) ) ) ) OR 

          ( ( x[3] == 2 ) AND ( ( ( x[0] == 1 ) AND ( ( ( x[1] == 0 ) AND ( K(3,0,2,[1, 0]) ) ) OR ( ( x[2] == 0 ) AND ( K(3,0,2,[2, 0]) ) ) ) ) OR ( ( x[1] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(3,0,2,[0, 1]) ) ) OR ( ( x[2] == 0 ) AND ( K(3,0,2,[2, 1]) ) ) ) ) OR ( ( x[2] == 1 ) AND ( ( ( x[0] == 0 ) AND ( K(3,0,2,[0, 2]) ) ) OR ( ( x[1] == 0 ) AND ( K(3,0,2,[1, 2]) ) ) ) ) ) ) 
     )

l[0][1]: # actual output, not cleaned 2
     ( 
          ( ( x[0] == 1 ) AND 
               ( ( x[0] == 0 ) AND 0 ) OR 
               ( ( x[1] == 0 ) AND K(0,0,1,[1]) ) OR 
               ( ( x[2] == 0 ) AND K(0,0,1,[2]) ) OR 
               ( ( x[3] == 0 ) AND K(0,0,1,[3]) ) ) 
          
          OR 
               
          ( ( x[1] == 1 ) AND ( ( x[0] == 0 ) AND K(1,0,1,[0]) ) OR ( ( x[1] == 0 ) AND 0 ) OR ( ( x[2] == 0 ) AND K(1,0,1,[2]) ) OR ( ( x[3] == 0 ) AND K(1,0,1,[3]) ) ) OR ( ( x[2] == 1 ) AND ( ( x[0] == 0 ) AND K(2,0,1,[0]) ) OR ( ( x[1] == 0 ) AND K(2,0,1,[1]) ) OR ( ( x[2] == 0 ) AND 0 ) OR ( ( x[3] == 0 ) AND K(2,0,1,[3]) ) ) OR ( ( x[3] == 1 ) AND ( ( x[0] == 0 ) AND K(3,0,1,[0]) ) OR ( ( x[1] == 0 ) AND K(3,0,1,[1]) ) OR ( ( x[2] == 0 ) AND K(3,0,1,[2]) ) OR ( ( x[3] == 0 ) AND 0 ) ) )


l[0][1]: # actual output, not cleaned 1
     (
          ( ( x[0] == 1 ) AND 
               ( ( x[0] == 1 ) AND 0 ) OR 
               ( ( x[1] == 1 ) AND K(0,0,1,[1]) ) OR 
               ( ( x[2] == 1 ) AND K(0,0,1,[2]) ) OR 
               ( ( x[3] == 1 ) AND K(0,0,1,[3]) ) ) 


          OR 

          
          ( ( x[1] == 1 ) AND 
               ( ( x[0] == 1 ) AND K(1,0,1,[0]) ) OR 
               ( ( x[1] == 1 ) AND 0 ) OR 
               ( ( x[2] == 1 ) AND K(1,0,1,[2]) ) OR 
               ( ( x[3] == 1 ) AND K(1,0,1,[3]) ) ) 
               
          
          OR 

          
          ( ( x[2] == 1 ) AND 
               ( ( x[0] == 1 ) AND K(2,0,1,[0]) ) OR 
               ( ( x[1] == 1 ) AND K(2,0,1,[1]) ) OR 
               ( ( x[2] == 1 ) AND 0 ) OR 
               ( ( x[3] == 1 ) AND K(2,0,1,[3]) ) ) 


          OR 
          

          ( ( x[3] == 1 ) AND 
               ( ( x[0] == 1 ) AND K(3,0,1,[0]) ) OR 
               ( ( x[1] == 1 ) AND K(3,0,1,[1]) ) OR 
               ( ( x[2] == 1 ) AND K(3,0,1,[2]) ) OR 
               ( ( x[3] == 1 ) AND 0 ) ) 
     )









def K_creator(b_first,w,bv):
     t = len(bv)

     if t == 0: return K(b_first,w,t,bv)

     b_last = bv[-1]
     new_bv = bv.copy()
     del new_bv[-1]
     new_bv = [b_first] + new_bv

     return K(b_last,w,t,new_bv)







l[0][1]:
     ( 
          ( ( x[0] == 0 ) AND 0 ) OR 
          ( ( x[1] == 0 ) AND k(1,0,1,0) ) OR 
          ( ( x[2] == 0 ) AND ( k(2,0,3,0) OR k(2,3,4,0) ) ) OR 
          ( ( x[3] == 0 ) AND ( k(3,0,3,0) OR k(3,3,4,0) )  ) OR
          
          ( ( x[0] == 1 ) AND K(0,0,1) ) OR 
          ( ( x[1] == 1 ) AND K(1,0,1) ) OR 
          ( ( x[2] == 1 ) AND K(2,0,1) ) OR 
          ( ( x[3] == 1 ) AND K(3,0,1) ) 
     ) 

l[0][2]:
     ( 
          ( l[0][1] ) OR 
          ( ( x[0] == 2 ) AND K(0,0,2) ) OR 
          ( ( x[1] == 2 ) AND K(1,0,2) ) OR 
          ( ( x[2] == 2 ) AND K(2,0,2) ) OR 
          ( ( x[3] == 2 ) AND K(3,0,2) ) 
     ) 

l[0][3]:
     ( 
          ( l[0][2] ) OR 
          ( ( x[0] == 3 ) AND K(0,0,3) ) OR 
          ( ( x[1] == 3 ) AND K(1,0,3) ) OR 
          ( ( x[2] == 3 ) AND K(2,0,3) ) OR 
          ( ( x[3] == 3 ) AND K(3,0,3) ) 
     )
          ( (x[0] == 1) AND 
               ( (x[0] == 0) AND (K(0,0,1,[0]) OR K(0,0,0,[])) ) OR
               ( (x[1] == 0) AND (K(1,0,0,[]) OR K(0,0,1,[1])) ) OR
               ( (x[2] == 0) AND (K(2,0,0,[]) OR K(0,0,1,[2])) ) OR
               ( (x[3] == 0) AND (K(3,0,0,[]) OR K(0,0,1,[3])) )
          
          OR
          
          ( (x[1] == 1) AND (x[0] == 0) AND (K(0,0,0,[]) OR K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,0,[]) OR K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(2,0,0,[]) OR K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(3,0,0,[]) OR K(1,0,1,[3])) )
          
          OR



          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 

l[0][1]: # Rewrite 4
     ( 
          ( (x[1] == 1) AND (x[0] == 0) AND (K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(1,0,1,[3])) )

          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 











l[0][1]:
     ( 
          ( ( x[0] == 0 ) AND 0 ) OR 
          ( ( x[1] == 0 ) AND k(1,0,1,0) ) OR 
          ( ( x[2] == 0 ) AND ( k(2,0,3,0) OR k(2,3,4,0) ) ) OR 
          ( ( x[3] == 0 ) AND ( k(3,0,3,0) OR k(3,3,4,0) )  ) OR
          
          ( ( x[0] == 1 ) AND K(0,0,1) ) OR 
          ( ( x[1] == 1 ) AND K(1,0,1) ) OR 
          ( ( x[2] == 1 ) AND K(2,0,1) ) OR 
          ( ( x[3] == 1 ) AND K(3,0,1) ) 
     ) 

l[0][2]:
     ( 
          ( l[0][1] ) OR 
          ( ( x[0] == 2 ) AND K(0,0,2) ) OR 
          ( ( x[1] == 2 ) AND K(1,0,2) ) OR 
          ( ( x[2] == 2 ) AND K(2,0,2) ) OR 
          ( ( x[3] == 2 ) AND K(3,0,2) ) 
     ) 

l[0][3]:
     ( 
          ( l[0][2] ) OR 
          ( ( x[0] == 3 ) AND K(0,0,3) ) OR 
          ( ( x[1] == 3 ) AND K(1,0,3) ) OR 
          ( ( x[2] == 3 ) AND K(2,0,3) ) OR 
          ( ( x[3] == 3 ) AND K(3,0,3) ) 
     )
          ( (x[0] == 1) AND 
               ( (x[0] == 0) AND (K(0,0,1,[0]) OR K(0,0,0,[])) ) OR
               ( (x[1] == 0) AND (K(1,0,0,[]) OR K(0,0,1,[1])) ) OR
               ( (x[2] == 0) AND (K(2,0,0,[]) OR K(0,0,1,[2])) ) OR
               ( (x[3] == 0) AND (K(3,0,0,[]) OR K(0,0,1,[3])) )
          
          OR
          
          ( (x[1] == 1) AND (x[0] == 0) AND (K(0,0,0,[]) OR K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,0,[]) OR K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(2,0,0,[]) OR K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(3,0,0,[]) OR K(1,0,1,[3])) )
          
          OR



          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 

l[0][1]: # Rewrite 4
     ( 
          ( (x[1] == 1) AND (x[0] == 0) AND (K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(1,0,1,[3])) )

          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 











l[0][1]:
     ( 
          ( ( x[0] == 0 ) AND 0 ) OR 
          ( ( x[1] == 0 ) AND k(1,0,1,0) ) OR 
          ( ( x[2] == 0 ) AND ( k(2,0,3,0) OR k(2,3,4,0) ) ) OR 
          ( ( x[3] == 0 ) AND ( k(3,0,3,0) OR k(3,3,4,0) )  ) OR
          
          ( ( x[0] == 1 ) AND K(0,0,1) ) OR 
          ( ( x[1] == 1 ) AND K(1,0,1) ) OR 
          ( ( x[2] == 1 ) AND K(2,0,1) ) OR 
          ( ( x[3] == 1 ) AND K(3,0,1) ) 
     ) 

l[0][2]:
     ( 
          ( l[0][1] ) OR 
          ( ( x[0] == 2 ) AND K(0,0,2) ) OR 
          ( ( x[1] == 2 ) AND K(1,0,2) ) OR 
          ( ( x[2] == 2 ) AND K(2,0,2) ) OR 
          ( ( x[3] == 2 ) AND K(3,0,2) ) 
     ) 

l[0][3]:
     ( 
          ( l[0][2] ) OR 
          ( ( x[0] == 3 ) AND K(0,0,3) ) OR 
          ( ( x[1] == 3 ) AND K(1,0,3) ) OR 
          ( ( x[2] == 3 ) AND K(2,0,3) ) OR 
          ( ( x[3] == 3 ) AND K(3,0,3) ) 
     )
          ( (x[0] == 1) AND 
               ( (x[0] == 0) AND (K(0,0,1,[0]) OR K(0,0,0,[])) ) OR
               ( (x[1] == 0) AND (K(1,0,0,[]) OR K(0,0,1,[1])) ) OR
               ( (x[2] == 0) AND (K(2,0,0,[]) OR K(0,0,1,[2])) ) OR
               ( (x[3] == 0) AND (K(3,0,0,[]) OR K(0,0,1,[3])) )
          
          OR
          
          ( (x[1] == 1) AND (x[0] == 0) AND (K(0,0,0,[]) OR K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,0,[]) OR K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(2,0,0,[]) OR K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(3,0,0,[]) OR K(1,0,1,[3])) )
          
          OR



          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 

l[0][1]: # Rewrite 4
     ( 
          ( (x[1] == 1) AND (x[0] == 0) AND (K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(1,0,1,[3])) )

          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 











l[0][1]:
     ( 
          ( ( x[0] == 0 ) AND 0 ) OR 
          ( ( x[1] == 0 ) AND k(1,0,1,0) ) OR 
          ( ( x[2] == 0 ) AND ( k(2,0,3,0) OR k(2,3,4,0) ) ) OR 
          ( ( x[3] == 0 ) AND ( k(3,0,3,0) OR k(3,3,4,0) )  ) OR
          
          ( ( x[0] == 1 ) AND K(0,0,1) ) OR 
          ( ( x[1] == 1 ) AND K(1,0,1) ) OR 
          ( ( x[2] == 1 ) AND K(2,0,1) ) OR 
          ( ( x[3] == 1 ) AND K(3,0,1) ) 
     ) 

l[0][2]:
     ( 
          ( l[0][1] ) OR 
          ( ( x[0] == 2 ) AND K(0,0,2) ) OR 
          ( ( x[1] == 2 ) AND K(1,0,2) ) OR 
          ( ( x[2] == 2 ) AND K(2,0,2) ) OR 
          ( ( x[3] == 2 ) AND K(3,0,2) ) 
     ) 

l[0][3]:
     ( 
          ( l[0][2] ) OR 
          ( ( x[0] == 3 ) AND K(0,0,3) ) OR 
          ( ( x[1] == 3 ) AND K(1,0,3) ) OR 
          ( ( x[2] == 3 ) AND K(2,0,3) ) OR 
          ( ( x[3] == 3 ) AND K(3,0,3) ) 
     )
          ( (x[0] == 1) AND 
               ( (x[0] == 0) AND (K(0,0,1,[0]) OR K(0,0,0,[])) ) OR
               ( (x[1] == 0) AND (K(1,0,0,[]) OR K(0,0,1,[1])) ) OR
               ( (x[2] == 0) AND (K(2,0,0,[]) OR K(0,0,1,[2])) ) OR
               ( (x[3] == 0) AND (K(3,0,0,[]) OR K(0,0,1,[3])) )
          
          OR
          
          ( (x[1] == 1) AND (x[0] == 0) AND (K(0,0,0,[]) OR K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,0,[]) OR K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(2,0,0,[]) OR K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(3,0,0,[]) OR K(1,0,1,[3])) )
          
          OR



          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 

l[0][1]: # Rewrite 4
     ( 
          ( (x[1] == 1) AND (x[0] == 0) AND (K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(1,0,1,[3])) )

          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 











l[0][1]:
     ( 
          ( ( x[0] == 0 ) AND 0 ) OR 
          ( ( x[1] == 0 ) AND k(1,0,1,0) ) OR 
          ( ( x[2] == 0 ) AND ( k(2,0,3,0) OR k(2,3,4,0) ) ) OR 
          ( ( x[3] == 0 ) AND ( k(3,0,3,0) OR k(3,3,4,0) )  ) OR
          
          ( ( x[0] == 1 ) AND K(0,0,1) ) OR 
          ( ( x[1] == 1 ) AND K(1,0,1) ) OR 
          ( ( x[2] == 1 ) AND K(2,0,1) ) OR 
          ( ( x[3] == 1 ) AND K(3,0,1) ) 
     ) 

l[0][2]:
     ( 
          ( l[0][1] ) OR 
          ( ( x[0] == 2 ) AND K(0,0,2) ) OR 
          ( ( x[1] == 2 ) AND K(1,0,2) ) OR 
          ( ( x[2] == 2 ) AND K(2,0,2) ) OR 
          ( ( x[3] == 2 ) AND K(3,0,2) ) 
     ) 

l[0][3]:
     ( 
          ( l[0][2] ) OR 
          ( ( x[0] == 3 ) AND K(0,0,3) ) OR 
          ( ( x[1] == 3 ) AND K(1,0,3) ) OR 
          ( ( x[2] == 3 ) AND K(2,0,3) ) OR 
          ( ( x[3] == 3 ) AND K(3,0,3) ) 
     )
          ( (x[0] == 1) AND 
               ( (x[0] == 0) AND (K(0,0,1,[0]) OR K(0,0,0,[])) ) OR
               ( (x[1] == 0) AND (K(1,0,0,[]) OR K(0,0,1,[1])) ) OR
               ( (x[2] == 0) AND (K(2,0,0,[]) OR K(0,0,1,[2])) ) OR
               ( (x[3] == 0) AND (K(3,0,0,[]) OR K(0,0,1,[3])) )
          
          OR
          
          ( (x[1] == 1) AND (x[0] == 0) AND (K(0,0,0,[]) OR K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,0,[]) OR K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(2,0,0,[]) OR K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(3,0,0,[]) OR K(1,0,1,[3])) )
          
          OR



          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 

l[0][1]: # Rewrite 4
     ( 
          ( (x[1] == 1) AND (x[0] == 0) AND (K(1,0,1,[0])) ) OR
          ( (x[1] == 1) AND (x[1] == 0) AND (K(1,0,1,[1])) ) OR
          ( (x[1] == 1) AND (x[2] == 0) AND (K(1,0,1,[2])) ) OR
          ( (x[1] == 1) AND (x[3] == 0) AND (K(1,0,1,[3])) )

          # ( x[1] == 1 ) AND ( x[1] == 0 ) ... OR # ---------- WRONG ----------
     ) 











l[0][1]:
     ( 
          ( ( x[0] == 0 ) AND 0 ) OR 
          ( ( x[1] == 0 ) AND k(1,0,1,0) ) OR 
          ( ( x[2] == 0 ) AND ( k(2,0,3,0) OR k(2,3,4,0) ) ) OR 
          ( ( x[3] == 0 ) AND ( k(3,0,3,0) OR k(3,3,4,0) )  ) OR
          
          ( ( x[0] == 1 ) AND K(0,0,1) ) OR 
          ( ( x[1] == 1 ) AND K(1,0,1) ) OR 
          ( ( x[2] == 1 ) AND K(2,0,1) ) OR 
          ( ( x[3] == 1 ) AND K(3,0,1) ) 
     ) 

l[0][2]:
     ( 
          ( l[0][1] ) OR 
          ( ( x[0] == 2 ) AND K(0,0,2) ) OR 
          ( ( x[1] == 2 ) AND K(1,0,2) ) OR 
          ( ( x[2] == 2 ) AND K(2,0,2) ) OR 
          ( ( x[3] == 2 ) AND K(3,0,2) ) 
     ) 

l[0][3]:
     ( 
          ( l[0][2] ) OR 
          ( ( x[0] == 3 ) AND K(0,0,3) ) OR 
          ( ( x[1] == 3 ) AND K(1,0,3) ) OR 
          ( ( x[2] == 3 ) AND K(2,0,3) ) OR 
          ( ( x[3] == 3 ) AND K(3,0,3) ) 
     )

SyntaxError: invalid syntax (1235548119.py, line 78)

In [ ]:
vector_k = k(2,2,4,1)
vector_k = clean_vector(vector_k)

print_vector(vector_k)

k(2,2,4,1) 
